# Generative AI Use Case: Summarize Dialogue

In [1]:
%pip install datasets
%pip install transformers
%pip install torch
%pip install torchdata


     -------------------------------------- 486.2/486.2 kB 3.8 MB/s eta 0:00:00
     -------------------------------------- 132.9/132.9 kB 3.8 MB/s eta 0:00:00
  Using cached dill-0.3.6-py3-none-any.whl (110 kB)
Note: you may need to restart the kernel to use updated packages.


The system cannot find the path specified.


The system cannot find the path specified.



Note: you may need to restart the kernel to use updated packages.


The system cannot find the path specified.


     ---------------------------------------- 1.3/1.3 MB 4.6 MB/s eta 0:00:00
     -------------------------------------- 172.4/172.4 MB 2.6 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0
    Uninstalling torch-2.0.0:
      Successfully uninstalled torch-2.0.0
Note: you may need to restart the kernel to use updated packages.


The system cannot find the path specified.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.15.1 requires torch==2.0.0, but you have torch 2.0.1 which is incompatible.
torchaudio 2.0.1 requires torch==2.0.0, but you have torch 2.0.1 which is incompatible.


In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import GenerationConfig

c:\Users\u336266\ANC\envs\nlp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
huggingface_dataset_name = "knkarthick/dialogsum"
dataset = load_dataset(huggingface_dataset_name)

Extracting data files: 100%|██████████| 3/3 [00:00<00:00, 118.42it/s]


Dataset csv downloaded and prepared to C:/Users/u336266/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-c07c4cf4362c223c/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<00:00, 286.06it/s]


In [4]:
example_indices = [40,200]
dash_line = '-'.join('' for x in range(100))

for i, index in enumerate(example_indices):
    print(dash_line)
    print("Example {}".format(i+1))
    print(dash_line)
    print('Input Dialogue:')
    print(dataset['test'][index]['dialogue'])
    print(dash_line)
    print('baseline summary:')
    print(dataset['test'][index]['summary'])
    print(dash_line)
    print()
    

---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
Input Dialogue:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
baseline summary:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------

---------------------------------------------------------------------------------------------------
Example 2


In [5]:
model_name = "google/flan-t5-base"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

c:\Users\u336266\ANC\envs\nlp\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\u336266\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

In [7]:
sentence = "Hello, my dog is cute"

sentence_encoded = tokenizer(sentence, return_tensors="pt")

sentence_decoded = tokenizer.decode(sentence_encoded['input_ids'][0], skip_special_tokens=True)

print("Encoded sentence: {}".format(sentence_encoded))
print("Decoded sentence: {}".format(sentence_decoded))


Encoded sentence: {'input_ids': tensor([[8774,    6,   82, 1782,   19, 5295,    1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}
Decoded sentence: Hello, my dog is cute


### Without any Prompt Engineering

In [8]:
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    inputs = tokenizer(dialogue, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens = 50,
        )[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print('Example ', i+1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{dialogue}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION WITHOUT PROMPT ENGINEERING:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
MODEL GENERATION WITHOUT PROMPT ENGINEERING:
Person1: It's ten to nine.

---------------------------------

### Zero Shot Inference with an Instruction Prompt

In [20]:
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    prompt = f"""
    Summarize the following conversation.

    {dialogue}

    Summary:
    """

    # Input constructed prompt instead of the dialogue
    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens = 50,
        )[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print('Example ', i+1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{dialogue}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION - ZERO SHOT:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
The train is about to leave.

----------------------------------------------


    Summarize the following conversation.

    #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

    Summary:
    

    Summarize the following conversation.

    #Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful